# Some more features

## Auxiliary data

In [1]:
import MDAnalysis as mda
from MDAnalysisTests.datafiles import PDB_sub_sol, XTC_sub_sol, XVG_BZ2

# Create your universe as usual
universe = mda.Universe(PDB_sub_sol, XTC_sub_sol)
# Attach an auxiliary time serie with the name `forces`
# In this example, the XVG file contains the force that applies to each atom.
universe.trajectory.add_auxiliary('forces', XVG_BZ2)
# Itarete through your trajectory, the time serie is kept in sync
for time_step in universe.trajectory:
    print(time_step.aux.forces)
# The first element of each array is the time in picoseconds.
# The next elements are the other columns of the XVG file.

[    0.        200.71288 -1552.2849  ...   128.4072   1386.0378
 -2699.3118 ]
[   50.      -1082.6454   -658.32166 ...  -493.02954   589.8844
  -739.2124 ]
[  100.       -246.27269   146.52911 ...   484.32501  2332.3767
 -1801.6234 ]


c:\users\dr19502\dev\mdanalysis\package\MDAnalysis\topology\PDBParser.py:315: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


## On-the-fly transformations

https://www.mdanalysis.org/2020/03/09/on-the-fly-transformations/

### Why?

In [2]:
import MDAnalysis as mda
import MDAnalysisData
import nglview as nv

peptide = MDAnalysisData.datasets.fetch_membrane_peptide()
u = mda.Universe(peptide.topology, peptide.trajectory)

_ColormakerRegistry()

In [4]:
nv.show_mdanalysis(u)

NGLWidget(max_frame=1000)

In [5]:
u.atoms.unwrap()
print(u.atoms.positions[[565, 566, 568, 841, 842, 843, 844, 845, 846], :])

u.trajectory[1]
u.trajectory[0]

print(u.atoms.positions[[565, 566, 568, 841, 842, 843, 844, 845, 846], :])

[[64.1896   39.550003 12.84    ]
 [64.459595 37.61     11.610001]
 [63.449596 38.380005 12.35    ]
 [66.959595 41.060005 16.18    ]
 [66.73959  41.38     18.490002]
 [67.5896   43.170006 17.2     ]
 [66.6096   42.070004 17.2     ]
 [65.1996   42.47     17.08    ]
 [64.5196   43.100006 15.870001]]
[[ 0.88000005 39.550003   12.84      ]
 [ 1.15       37.61       11.610001  ]
 [ 0.14       38.380005   12.35      ]
 [ 3.65       41.060005   16.18      ]
 [ 3.4300003  41.38       18.490002  ]
 [ 4.28       43.170006   17.2       ]
 [ 3.3000002  42.070004   17.2       ]
 [ 1.8900001  42.47       17.08      ]
 [ 1.21       43.100006   15.870001  ]]


### Setting a workflow

In [6]:
import MDAnalysis as mda
from MDAnalysis import transformations
import MDAnalysisData
import nglview as nv

peptide = MDAnalysisData.datasets.fetch_membrane_peptide()
u = mda.Universe(peptide.topology, peptide.trajectory)
workflow = [transformations.unwrap(u.atoms)]
u.trajectory.add_transformations(*workflow)

In [7]:
nv.show_mdanalysis(u)

NGLWidget(max_frame=1000)

<p style="color: red; font-size: 300%;">The transformation workflow can only be set once!</h1>

## Combine transformations

In [8]:
import MDAnalysis as mda
from MDAnalysis import transformations
import MDAnalysisData
import nglview as nv

peptide = MDAnalysisData.datasets.fetch_membrane_peptide()
u = mda.Universe(peptide.topology, peptide.trajectory)
helix = u.select_atoms('protein')
workflow = [
    transformations.unwrap(u.atoms),
    transformations.center_in_box(helix),
    transformations.wrap(u.atoms, compound='fragments')
]
u.trajectory.add_transformations(*workflow)

In [9]:
nv.show_mdanalysis(u)

NGLWidget(max_frame=1000)

### Transformations out of the box

* translation
* center in box
* rotation
* average position
* fit (translation)
* fit (translation and rotation)
* wrap
* unwrap

### Writing a transformation

In [10]:
import MDAnalysis as mda
from MDAnalysis import transformations
import MDAnalysisData
import nglview as nv

def up_by_40(ts):
    ts.positions += [0, 0, 40]
    return ts


peptide = MDAnalysisData.datasets.fetch_membrane_peptide()
u = mda.Universe(peptide.topology, peptide.trajectory)
workflow = [up_by_40]
u.trajectory.add_transformations(*workflow)

In [11]:
v = nv.show_mdanalysis(u)
v.add_unitcell()
v

NGLWidget(max_frame=1000)

In [14]:
class UpByX:
    def __init__(self, x):
        self.x = x
    
    def __call__(self, ts):
        ts.positions += [0, 0, self.x]
        return ts
    
peptide = MDAnalysisData.datasets.fetch_membrane_peptide()
u = mda.Universe(peptide.topology, peptide.trajectory)
workflow = [UpByX(20)]
u.trajectory.add_transformations(*workflow)

In [15]:
v = nv.show_mdanalysis(u)
v.add_unitcell()
v

NGLWidget(max_frame=1000)